In [17]:
from pathlib import Path
import polars as pl

In [ ]:
# Assuming either all *_compressed.tar.zst files or all_groups_lean.tar.zst file is/are decompressed
CWD = Path().resolve()
BASE = CWD.parent

print(f"Folder where group folder should be: {BASE}")
print(f"Current working directory (where table will be saved): {CWD}")

Folder where group folder should be: /data/users/bdupin/datasets-pnas
Current working directory (where table will be saved): /data/users/bdupin/datasets-pnas/code


In [19]:
GROUP_ORDER = [
    "fungi_mit",
    "metazoans_mit",
    "plants_mit",
    "plants_plt",
    "protists_mit",
    "protists_plt",
    "green_algae_mit",
    "green_algae_plt",
]

label_map = {
    "fungi_mit": "Fungi (mitochondria)",
    "green_algae_mit": "Green algae (mitochondria)",
    "metazoans_mit": "Metazoans (mitochondria)",
    "plants_mit": "Plants (mitochondria)",
    "protists_mit": "Protists (mitochondria)",
    "green_algae_plt": "Green algae (plastid)",
    "plants_plt": "Plants (plastid)",
    "protists_plt": "Protists (plastid)",
}

polarity_map = {"++": "same", "--": "same", "+-": "opposite", "-+": "opposite"}

MIN_IGR = 40
TAIL_FRACTION = 0.1

###  extraction of brms/PGLMM results

In [ ]:
brms_effects = {}
for g in GROUP_ORDER:
    print(f"Processing group: {g}")
    brm_result = BASE / g / "brms_result" / "brms_result.txt"
    for line in brm_result.read_text().splitlines():
        ls = line.strip()
        if '[1] "Delta PGLMM =' in ls:
            b = round(float(ls.split("= ")[1].split(" ", 1)[0].strip()), 3)
            lo = round(float(ls.split("(")[1].split("-")[0].strip()), 3)
            hi = round(float(ls.split("-")[1].split(")")[0].strip()), 3)

            fold = round((10**b), 3)
            fold_lo = round((10**lo), 3)
            fold_hi = round((10**hi), 3)

            brms_effects[g] = {
                "est": b,
                "lo": lo,
                "hi": hi,
                "fold": fold,
                "fold_lo": fold_lo,
                "fold_hi": fold_hi,
            }

            print(f"Extracted from brms result: b={b}, lo={lo}, hi={hi}")
            print(f" Fold change: {fold} (CI: {fold_lo} - {fold_hi})")

Processing group: fungi_mit
Extracted from brms result: b=0.473, lo=0.44, hi=0.507
Fold change: 2.972 (CI: 2.754 - 3.214)
Processing group: metazoans_mit
Extracted from brms result: b=0.38, lo=0.374, hi=0.386
Fold change: 2.399 (CI: 2.366 - 2.432)
Processing group: plants_mit
Extracted from brms result: b=0.522, lo=0.509, hi=0.536
Fold change: 3.327 (CI: 3.228 - 3.436)
Processing group: plants_plt
Extracted from brms result: b=0.292, lo=0.29, hi=0.293
Fold change: 1.959 (CI: 1.95 - 1.963)
Processing group: protists_mit
Extracted from brms result: b=0.385, lo=0.362, hi=0.409
Fold change: 2.427 (CI: 2.301 - 2.564)
Processing group: protists_plt
Extracted from brms result: b=0.303, lo=0.297, hi=0.309
Fold change: 2.009 (CI: 1.982 - 2.037)
Processing group: green_algae_mit
Extracted from brms result: b=0.193, lo=0.143, hi=0.244
Fold change: 1.56 (CI: 1.39 - 1.754)
Processing group: green_algae_plt
Extracted from brms result: b=0.224, lo=0.208, hi=0.239
Fold change: 1.675 (CI: 1.614 - 1.734

### Descriptive statistics of intergenic regions

In [21]:
global_true = 0
global_false = 0
global_total = 0

support_info = {}

for g in GROUP_ORDER:
    igs = pl.read_csv(BASE / g / "summary_igs_intergenic.tsv", separator="\t")

    igs = igs.with_columns(
        pl.col("Polarity")
        .replace_strict(polarity_map)
        .cast(pl.Categorical)
        .alias("polarity_bin")
    )

    med = igs.group_by(["AN", "polarity_bin"]).agg(
        pl.col("Length").median().alias("median_bp")
    )

    wide = med.pivot(values="median_bp", index="AN", on="polarity_bin")
    wide = wide.with_columns((pl.col("opposite") > pl.col("same")).alias("support_opp"))

    group_true = wide.filter(pl.col("support_opp")).height
    group_false = wide.filter(~pl.col("support_opp")).height

    global_true += group_true
    global_false += group_false
    global_total += group_true + group_false

    support_info[g] = {
        "true": group_true,
        "false": group_false,
        "total_genomes": group_true + group_false,
    }

    print(f"Within Group {g}, out of {group_true + group_false} genomes:")
    print(
        f" Percent  True: {group_true / (group_true + group_false) * 100:.2f}% ({group_true})"
    )
    print(
        f" Percent False: {group_false / (group_true + group_false) * 100:.2f}% ({group_false})"
    )
    print("")

print(f"Overall Results, out of {global_total} genomes:")
print(
    f" Percent  True: {global_true / (global_true + global_false) * 100:.2f}% ({global_true})"
)
print(
    f" Percent False: {global_false / (global_true + global_false) * 100:.2f}% ({global_false})"
)

Within Group fungi_mit, out of 424 genomes:
 Percent  True: 82.78% (351)
 Percent False: 17.22% (73)

Within Group metazoans_mit, out of 18011 genomes:
 Percent  True: 68.49% (12335)
 Percent False: 31.51% (5676)

Within Group plants_mit, out of 1038 genomes:
 Percent  True: 94.41% (980)
 Percent False: 5.59% (58)

Within Group plants_plt, out of 17525 genomes:
 Percent  True: 99.93% (17512)
 Percent False: 0.07% (13)

Within Group protists_mit, out of 689 genomes:
 Percent  True: 80.41% (554)
 Percent False: 19.59% (135)

Within Group protists_plt, out of 704 genomes:
 Percent  True: 98.86% (696)
 Percent False: 1.14% (8)

Within Group green_algae_mit, out of 95 genomes:
 Percent  True: 75.79% (72)
 Percent False: 24.21% (23)

Within Group green_algae_plt, out of 270 genomes:
 Percent  True: 88.52% (239)
 Percent False: 11.48% (31)

Overall Results, out of 38756 genomes:
 Percent  True: 84.47% (32739)
 Percent False: 15.53% (6017)


### Enrichment analysis

In [ ]:
def genome_enrichment_stats(df_g: pl.DataFrame, tail_frac: float) -> dict:
    n = df_g.height

    # Count polarities
    n_opp = df_g.filter(pl.col("polarity_bin") == "opposite").height
    n_same = df_g.filter(pl.col("polarity_bin") == "same").height

    # Δlog10 (opp - same) - computed using median(log10(length))
    median_opp = (
        df_g.filter(pl.col("polarity_bin") == "opposite")
        .select(pl.col("Length").log10().median())
        .item()
    )
    median_same = (
        df_g.filter(pl.col("polarity_bin") == "same")
        .select(pl.col("Length").log10().median())
        .item()
    )

    delta_log10 = median_opp - median_same
    fold_change = 10**delta_log10

    tail_size = int(round(tail_frac * n))

    # Sort by length and get top/bottom tails
    df_sorted = df_g.sort("Length", descending=True)
    top_tail = df_sorted.head(tail_size)
    bottom_tail = df_sorted.tail(tail_size)

    # Calculate fractions
    frac_opp_all = n_opp / n
    if frac_opp_all == 0:
        raise ValueError(
            f"Fraction of opposite polarity is zero, thefore AN {df_g['AN'][0]} is monopolar!"
        )

    frac_opp_top = (
        top_tail.filter(pl.col("polarity_bin") == "opposite").height / tail_size
    )

    frac_opp_bottom = (
        bottom_tail.filter(pl.col("polarity_bin") == "opposite").height / tail_size
    )

    # Enrichment ratios
    enrich_top = frac_opp_top / frac_opp_all
    depletion_botton = frac_opp_bottom / frac_opp_all

    return {
        "AN": df_g["AN"][0],
        "n_igr": n,
        "n_opp": n_opp,
        "n_same": n_same,
        "tail_size": tail_size,
        "frac_opp_all": frac_opp_all,
        "frac_opp_top": frac_opp_top,
        "frac_opp_bottom": frac_opp_bottom,
        "enrich_top": enrich_top,
        "depletion_botton": depletion_botton,
        "delta_log10": delta_log10,
        "fold_change": fold_change,
    }

In [ ]:
enrich_path = BASE / "code" / "enrich_data"
enrich_path.mkdir(exist_ok=True)
group_enrich = {}
enrich_for_table = {}

for g in GROUP_ORDER:
    print(f"Processing group: {g}")
    igs = pl.scan_csv(BASE / g / "summary_igs_intergenic.tsv", separator="\t")

    igs = igs.with_columns(
        pl.col("Polarity")
        .replace_strict(polarity_map)
        .cast(pl.Categorical)
        .alias("polarity_bin")
    ).collect()

    an_counts = igs.group_by("AN").agg(pl.len().alias("count"))
    valid_ans = an_counts.filter(pl.col("count") >= MIN_IGR)["AN"].to_list()
    igs = igs.filter(pl.col("AN").is_in(valid_ans))

    rows = []
    for subset in igs.partition_by("AN", maintain_order=True):
        data = genome_enrichment_stats(
            subset,
            tail_frac=TAIL_FRACTION,
        )
        rows.append(data)

    per_genome = pl.DataFrame([r for r in rows if r])

    # Sort by AN and round float columns
    per_genome = per_genome.sort("AN").with_columns(pl.col(pl.Float64).round(3))

    # ties (i.e., 1.0) are considered not enriched/depleted, so we use > and < to create boolean columns
    per_genome = per_genome.with_columns(
        (pl.col("enrich_top") > 1.0).alias("top_enriched"),
        (pl.col("depletion_botton") < 1.0).alias("bottom_depleted"),
    )

    file_out = enrich_path / f"{g}_enrichment.tsv"
    per_genome.write_csv(file_out, separator="\t")
    per_genome = per_genome.with_columns(pl.lit(g).alias("group"))
    group_enrich[g] = per_genome

    n_genomes = per_genome.height
    n_top_enriched = per_genome.filter(pl.col("top_enriched")).height
    n_bottom_depleted = per_genome.filter(pl.col("bottom_depleted")).height

    enrich_for_table[g] = {
        "genomes_over_40_igr": n_genomes,
        "top_enriched_pct": round((n_top_enriched / n_genomes) * 100, 2),
        "top_enriched_n": n_top_enriched,
        "bottom_depleted_pct": round((n_bottom_depleted / n_genomes) * 100, 2),
        "bottom_depleted_n": n_bottom_depleted,
    }

    print(f" Results for {len(per_genome)} genomes written to {file_out}")
    print()

Processing group: fungi_mit
 Results for 253 genomes written to /data/users/bdupin/datasets-pnas/code/enrich_data/fungi_mit_enrichment.tsv

Processing group: metazoans_mit
 Results for 16 genomes written to /data/users/bdupin/datasets-pnas/code/enrich_data/metazoans_mit_enrichment.tsv

Processing group: plants_mit
 Results for 750 genomes written to /data/users/bdupin/datasets-pnas/code/enrich_data/plants_mit_enrichment.tsv

Processing group: plants_plt
 Results for 17473 genomes written to /data/users/bdupin/datasets-pnas/code/enrich_data/plants_plt_enrichment.tsv

Processing group: protists_mit
 Results for 567 genomes written to /data/users/bdupin/datasets-pnas/code/enrich_data/protists_mit_enrichment.tsv

Processing group: protists_plt
 Results for 698 genomes written to /data/users/bdupin/datasets-pnas/code/enrich_data/protists_plt_enrichment.tsv

Processing group: green_algae_mit
 Results for 77 genomes written to /data/users/bdupin/datasets-pnas/code/enrich_data/green_algae_mit_

In [ ]:
df_all = pl.concat(group_enrich.values())
df_all = df_all.select(
    ["AN", "group"] + [col for col in df_all.columns if col not in ["AN", "group"]]
)
df_all = df_all.sort(["group", "AN"])
df_all.write_csv(enrich_path / "all_groups_enrichment.tsv", separator="\t")

## Table 1 Generation

In [26]:
rows = []

for g in GROUP_ORDER:
    info = support_info[g]
    brm = brms_effects[g]
    enrich = enrich_for_table[g]

    rows.append(
        {
            "Taxonomic group and organelle type": label_map[g],
            "Total Genomes": info["total_genomes"],
            "% med(opposite) > med(same)": round(
                (info["true"] / info["total_genomes"]) * 100, 2
            ),
            "Δ PGLMM 95% CI": f"{brm['est']} ({brm['lo']} - {brm['hi']})",
            "fold PGLMM 95% CI": f"{brm['fold']} ({brm['fold_lo']} - {brm['fold_hi']})",
            "Genomes with >= 40 IGRs": enrich["genomes_over_40_igr"],
            "% Top Enriched (count)": f"{enrich['top_enriched_pct']}% ({enrich['top_enriched_n']})",
            "% Bottom Depleted (count)": f"{enrich['bottom_depleted_pct']}% ({enrich['bottom_depleted_n']})",
        }
    )

table1 = pl.DataFrame(rows)
table1.write_csv(BASE / "code" / "table1.tsv", separator="\t")